In [1]:
import time
#time.sleep(60*60*3)
import os
os.environ["TF_GPU_THREAD_MODE"] = "gpu_private"

In [2]:
import os
# --- Uncomment to use only CPU (e.g. GPU memory is too small)
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU') 
gpu_devices = tf.config.experimental.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(physical_devices[0], True)
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

    
from tensorflow import keras
from tensorflow.python.data import AUTOTUNE

    
tf.test.is_gpu_available(cuda_only=True) 

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [3]:
# reate logger - nicer formatting
import logging

logger = logging.getLogger()
logging._warn_preinit_stderr = 0
logger.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s: %(message)s')
ch = logging.StreamHandler()
ch.setFormatter(formatter)
logger.handlers = [ch]
logger.info("Logger started.")

2021-03-06 12:18:01,616: Logger started.


In [4]:
# Coppied from pickle2sqlite.ipynb


import itertools
import math
import os
import pickle
import numpy as np


def angle_to_sin_cos(angle):
    return [math.sin(angle), math.cos(angle)]


def sin_cos_to_angle(sin, cos):
    return math.atan2(sin, cos)

def angle_to_sin_and_back(angle):
    return math.atan2(*angle_to_sin_cos(angle))

def to_abs_with_sin_cos(arr):
    result = list()
    for box in arr:
        angles = [angle_to_sin_cos(angle) for angle in box[3:]]
        result.append(list(box[:3]) + list(itertools.chain(*angles)))
    return np.asarray(result)

def process_abs_sample(sample):
    """ Changes sample in form of single 6 elements array to houdini compatible vec3 pos and vec4 orient """
    orient = [sin_cos_to_angle(sample[3+2*x], sample[4+2*x]) for x in range(3)] + [1]
    return sample[:3], orient

class DataProcessor(object):
    """ Code responsible for processing of data gathered into a class for readability.

    Trying few different approaches to data representation to test what works best:

    For every frame n following representations are calculated:
    - x_delta: array (4, 6) of delta between frames n and n-1: (Δx, Δy, Δz, Δα , Δβ, Δγ)
    - x_abs_n_minus_1: array (2, 4, 9) of absolute values in frame n-1
    - x_abs_n: x_abs in in frame n
      Structure of x_abs_*: [[x, y, z, sin(α), cos(α), sin(β), cos(β), sin(γ), cos(γ)], [...]]
    - x_force: array of forces in frame n: (power, sin(Φ), cos(Φ)

    - y_delta: array (4, 6) of delta between frames n and n+1
    - y_abs: array (4, 9) of absolute values in frame n+1

    Data is passed to neural networks in forms:
    - x: (x_delta, x_force) y: y_delta
    - x: (x_delta, x_abs_n, x_force) y: y_delta
    - x: (x_delta, x_abs_n, x_force) y: y_abs
    - x: (x_abs_n_minus_1, x_abs_n, x_force) y: y_abs
    - x: (x_abs_n_minus_1, x_abs_n, x_force) y: y_delta
    """
    def __init__(self, paths):
        # self.x_delta = np.empty((1, 4, 6))
        # self.x_abs_n_minus_1 = np.empty((1, 4, 9))
        # self.x_abs_n = np.empty((1, 4, 9))
        # self.x_force = np.empty((1, 3))
        #
        # self.y_delta = np.empty((1, 4, 6))
        # self.y_abs = np.empty((1, 4, 9))

        self.x_delta = list()
        self.x_abs_n_minus_1 = list()
        self.x_abs_n = list()
        self.x_force = list()

        self.y_delta = list()
        self.y_abs = list()

        for num, file_path in enumerate(paths):
            print("{} / {}".format(num, len(file_paths)), end="\r")
            with open(file_path, "rb") as f:
                frames = pickle.load(f)
                filename = os.path.basename(file_path)
                force, force_angle = [float(part) for part in filename.split("_")]
                last_frame = None
                for num, (frame, next_frame) in enumerate(zip(frames[:-1], frames[1:])):
                    # print(num, end="\r")
                    if num <=2:
                        frame_force = 0
                    else:
                        frame_force = force
                    if last_frame is not None:
                        x_force = np.asarray(angle_to_sin_cos(force_angle) + [frame_force])
                        n_minus_1_abs = np.asarray([f[0] + f[1][:-1] for f in last_frame])
                        n_abs = np.asarray([f[0] + f[1][:-1] for f in frame])
                        n_plus_1_abs = np.asarray([f[0] + f[1][:-1] for f in next_frame])

                        x_delta = n_abs - n_minus_1_abs
                        y_delta = n_plus_1_abs - n_abs

                        n_minus_1_abs = to_abs_with_sin_cos(n_minus_1_abs)
                        n_abs = to_abs_with_sin_cos(n_abs)
                        y_abs = to_abs_with_sin_cos(n_plus_1_abs)

                        self.x_delta.append(x_delta)
                        self.x_abs_n_minus_1.append(n_minus_1_abs)
                        self.x_abs_n.append(n_abs)
                        self.x_force.append(x_force)

                        self.y_delta.append(y_delta)
                        self.y_abs.append(y_abs)
                    last_frame = frame

        self.x_delta = np.asarray(self.x_delta)
        self.x_abs_n_minus_1 = np.asarray(self.x_abs_n_minus_1)
        self.x_abs_n = np.asarray(self.x_abs_n)
        self.x_force = np.asarray(self.x_force)

        self.y_delta = np.asarray(self.y_delta)
        self.y_abs = np.asarray(self.y_abs)

In [5]:
from tensorflow.data import Dataset


data_dir = os.path.normpath("samplesGeneration/data")
file_names = os.listdir(data_dir)
file_paths = [os.path.join(data_dir, file_name) for file_name in file_names]
dp = DataProcessor(file_paths)

In [6]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
early_stopping = EarlyStopping(monitor='val_loss', patience=4)


models_dir = os.path.join(os.getcwd(), "samplesGeneration", "logs")

In [7]:
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Reshape, Concatenate
from tensorflow.keras.layers import LSTM, GRU
import time
import tensorflow.keras.backend as K


def special_loss(y_true, y_pred):
    custom_loss=tf.add(tf.square(y_true-y_pred)/tf.add(tf.square(y_pred), 0.1), tf.k.losses.MSE(y_true, y_pred))
    return custom_loss

def loss(y_true,y_pred):
    return K.sum(K.square(y_pred - y_true) * K.square(100 * y_true) + K.square(y_pred - y_true))

# physical_devices = tf.config.experimental.list_physical_devices('GPU')
# tf.config.experimental.set_memory_growth(physical_devices[0], True)

def create_datasets(dp, y, batch_size=32):
    number_of_samples = len(dp.x_abs_n)
    indexes = np.arange(number_of_samples)
    np.random.shuffle(indexes)
    split = int(number_of_samples * 0.9)
    val_length = number_of_samples - split
    train_indexes = indexes[:split]
    val_indexes = indexes[split:]

    ds = tf.data.Dataset.from_tensor_slices(({"x_abs_n_minus_1": dp.x_abs_n_minus_1[train_indexes].reshape(split, 1, 4, -1),
                                              "x_abs_n": dp.x_abs_n[train_indexes].reshape(split, 1, 4, -1),
                                              "x_delta": dp.x_delta[train_indexes].reshape(split, 1, 4, -1),
                                              "x_force": dp.x_force[train_indexes].reshape(split, 1, -1),
                                              }, y[train_indexes].reshape(split, 1, 4, -1)))
    ds.cache()
    ds.shuffle(buffer_size=1000)
    ds.batch(batch_size, drop_remainder=True)
    ds.prefetch(buffer_size=AUTOTUNE)

    val_ds = tf.data.Dataset.from_tensor_slices(({"x_abs_n_minus_1": dp.x_abs_n_minus_1[val_indexes].reshape(val_length, 1, 4, -1),
                                              "x_abs_n": dp.x_abs_n[val_indexes].reshape(val_length, 1, 4, -1),
                                              "x_delta": dp.x_delta[val_indexes].reshape(val_length, 1, 4, -1),
                                              "x_force": dp.x_force[val_indexes].reshape(val_length, 1, -1),
                                              }, y[val_indexes].reshape(val_length, 1, 4, -1)))
    val_ds.cache()
    val_ds.shuffle(buffer_size=1000)
    val_ds.batch(batch_size, drop_remainder=True)
    val_ds.prefetch(buffer_size=AUTOTUNE)

    return ds, val_ds

def create_and_train_network(dp, y, model_id=0, epohs=10, batch_size=32, model_generator=None):
    y_shape = y[0].shape
    log_dir = os.path.join(os.getcwd(), "samplesGeneration", "logs_2", "model_{}_{}_{}_{}".format(y.shape, y_shape, model_id, time.time()))
    if not os.path.isdir(log_dir):
        os.makedirs(log_dir)
    tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1, profile_batch = '500,520')

    best_model_file = "{}_{}_{}_model.h5".format(y.shape, y_shape, model_id)
    best_model = ModelCheckpoint(best_model_file, monitor='val_loss', mode='min',verbose=1, save_best_only=True)

    class CustomSaveCallback(keras.callbacks.Callback):

        def on_test_begin(self, logs=None):
            self.model.save("model\\model_{}_{}_{}_{}".format(y.shape, y_shape, model_id, time.time()), save_format='tf')
    my_cb = CustomSaveCallback()
    train_dataset, validation_dataset =  create_datasets(dp, y, batch_size)

    def create_model0():
        input_delta = Input(shape=(4, 6), name='x_delta')
        input_abs = Input(shape=(4, 9), name='x_abs_n')
        input_force = Input(shape=(3, 1), name='x_force')

        input_delta_flat = Flatten()(input_delta)
        input_abs_flat = Flatten()(input_abs)
        input_force_flat = Flatten()(input_force)

        x = Concatenate()([input_delta_flat, input_abs_flat, input_force_flat])
        x = Dense(64, activation='relu')(x)
        x = Dropout(0.2)(x)
        x = Dense(128, activation='relu')(x)
        x = Dropout(0.2)(x)
        x = Dense(128, activation='relu')(x)
        x = Dropout(0.2)(x)
        x = Dense(np.prod(y_shape))(x)

        output_1 = Reshape(y_shape)(x)
        model = Model(inputs=[input_delta, input_abs, input_force], outputs=[output_1])
        model.compile(loss="mse", optimizer='rmsprop')
        return model

    def create_model1():
        input_delta = Input(shape=(4, 6), name='x_delta')
        input_abs = Input(shape=(4, 9), name='x_abs_n')
        input_force = Input(shape=(3, 1), name='x_force')

        input_delta_flat = Flatten()(input_delta)
        input_abs_flat = Flatten()(input_abs)
        input_force_flat = Flatten()(input_force)

        x = Dense(64, activation='relu')(input_force_flat)
        x = Dropout(0.2)(x)
        x = Concatenate()([x, input_delta_flat])
        x = Dense(128, activation='relu')(x)
        x = Dropout(0.2)(x)
        x = Concatenate()([x, input_abs_flat])
        x = Dense(128, activation='relu')(x)
        x = Dropout(0.2)(x)
        x = Dense(np.prod(y_shape))(x)

        output_1 = Reshape(y_shape)(x)
        model = Model(inputs=[input_delta, input_abs, input_force], outputs=[output_1])
        model.compile(loss="mse", optimizer='rmsprop')
        return model

    def create_model2():
        input_delta = Input(shape=(4, 6), name='x_delta')
        input_abs = Input(shape=(4, 9), name='x_abs_n')
        input_force = Input(shape=(3, 1), name='x_force')

        input_delta_flat = Flatten()(input_delta)
        input_abs_flat = Flatten()(input_abs)
        input_force_flat = Flatten()(input_force)

        x = Concatenate()([input_delta_flat, input_abs_flat, input_force_flat])
        x = Dense(128, activation='relu')(x)
        x = Dropout(0.2)(x)
        x = Dense(128, activation='relu')(x)
        x = Dropout(0.2)(x)
        x = Dense(64, activation='relu')(x)
        x = Dropout(0.2)(x)
        x = Dense(64, activation='relu')(x)
        x = Dropout(0.2)(x)
        x = Dense(np.prod(y_shape))(x)

        output_1 = Reshape(y_shape)(x)
        model = Model(inputs=[input_delta, input_abs, input_force], outputs=[output_1])
        model.compile(loss="mse", optimizer='rmsprop')
        return model

    def create_model3():
        input_delta = Input(shape=(4, 6), name='x_delta')
        input_abs_0 = Input(shape=(4, 9), name='x_abs_n_minus_1')
        input_abs = Input(shape=(4, 9), name='x_abs_n')
        input_force = Input(shape=(3, 1), name='x_force')

        input_abs_0_flat = Flatten()(input_abs_0)
        input_abs_flat = Flatten()(input_abs)
        input_force_flat = Flatten()(input_force)
        x = Concatenate()([input_abs_0_flat, input_abs_flat])
        x = Reshape((2, 4*9))(x)
        x = LSTM(np.prod(y_shape), return_sequences=False, dropout=0.2,
        activation="tanh", recurrent_activation="sigmoid", recurrent_dropout=0, unroll=False, use_bias=True
                )(x)
        x = Dense(np.prod(y_shape))(x)

        output_1 = Reshape(y_shape)(x)
        model = Model(inputs=[input_abs_0, input_abs, input_force], outputs=[output_1])
        model.compile(loss="mse", optimizer='rmsprop')
        return model

    def create_model4():
        input_delta = Input(shape=(4, 6), name='x_delta')
        input_abs_0 = Input(shape=(4, 9), name='x_abs_n_minus_1')
        input_abs = Input(shape=(4, 9), name='x_abs_n')
        input_force = Input(shape=(3, 1), name='x_force')

        input_abs_0_flat = Flatten()(input_abs_0)
        input_abs_flat = Flatten()(input_abs)
        input_force_flat = Flatten()(input_force)
        x = Concatenate()([input_abs_0_flat, input_abs_flat])
        x = Reshape((2, 4*9))(x)
        x = GRU(np.prod(y_shape), return_sequences=False,
        activation="tanh", recurrent_activation="sigmoid", recurrent_dropout=0, unroll=False, use_bias=True, reset_after=True
        )(x)
        x = Flatten()(x)
        x = Concatenate()([x, input_force_flat])
        x = Dropout(0.2)(x)
        x = Dense(np.prod(y_shape))(x)

        output_1 = Reshape(y_shape)(x)
        model = Model(inputs=[input_abs_0, input_abs, input_force], outputs=[output_1])
        model.compile(loss="mse", optimizer='rmsprop')
        return model

    logger.info("Create model...")
    if model_generator is not None:
        model = model_generator()
    elif model_id == 0:
        model = create_model0()
    elif model_id == 1:
        model = create_model1()
    elif model_id == 2:
        model = create_model2()
    elif model_id == 3:
        model = create_model3()
    elif model_id == 4:
        model = create_model4()
    model.summary()
    logger.info("Fit model..")
    network = model.fit(train_dataset, validation_data=validation_dataset, verbose=1,
                        epochs=epohs, callbacks=[my_cb, early_stopping, best_model, tensorboard_callback])
    logger.info("Summary")
    model.summary()

    models_dir = os.path.join(os.getcwd(), "samplesGeneration", "logs", "model_{}".format(time.time()))
    model.save(os.path.join(models_dir, "model"))
    return network

logger.info("start")

#training_generator = tf.data.Dataset.from_generator(lambda: training_generator,
#                                                    output_types=(tf.float32, tf.float32))
#validation_generator = tf.data.Dataset.from_generator(lambda: training_generator,
#                                                    output_types=(tf.float32, tf.float32))

2021-03-06 12:18:41,890: start


In [8]:
# create_and_train_network(dp, dp.y_delta, 0)
# create_and_train_network(dp, dp.y_delta, 1)
# create_and_train_network(dp, dp.y_delta, 2)

In [9]:

# create_and_train_network(dp, dp.y_delta, 1)

In [10]:

#create_and_train_network(dp, dp.y_abs, 0)
#create_and_train_network(dp, dp.y_abs, 1)

In [11]:
#create_and_train_network(dp, dp.y_abs, 4)

In [12]:
#create_and_train_network(dp, dp.y_delta, 4)

In [13]:
def create_model2_improved():
    input_delta = Input(shape=(4, 6), name='x_delta')
    input_abs = Input(shape=(4, 9), name='x_abs_n')
    input_force = Input(shape=(3, 1), name='x_force')

    input_delta_flat = Flatten()(input_delta)
    input_abs_flat = Flatten()(input_abs)
    input_force_flat = Flatten()(input_force)

    x = Concatenate()([input_delta_flat, input_abs_flat, input_force_flat])
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(2048, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(np.prod(dp.y_delta[1].shape))(x)

    output_1 = Reshape(dp.y_delta[1].shape)(x)
    model = Model(inputs=[input_delta, input_abs, input_force], outputs=[output_1])
    model.compile(loss="mse", optimizer='rmsprop')
    return model
# 0.0091
# create_and_train_network(dp, dp.y_delta, 10, model_generator=create_model2_improved)

In [14]:
def create_model2_improved():
    input_delta = Input(shape=(4, 6), name='x_delta')
    input_abs = Input(shape=(4, 9), name='x_abs_n')
    input_force = Input(shape=(3, 1), name='x_force')

    input_delta_flat = Flatten()(input_delta)
    input_abs_flat = Flatten()(input_abs)
    input_force_flat = Flatten()(input_force)

    x = Concatenate()([input_delta_flat, input_abs_flat, input_force_flat])
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(np.prod(dp.y_abs[1].shape))(x)

    output_1 = Reshape(dp.y_abs[1].shape)(x)
    model = Model(inputs=[input_delta, input_abs, input_force], outputs=[output_1])
    model.compile(loss="mse", optimizer='rmsprop')
    return model
#create_and_train_network(dp, dp.y_abs, 5, model_generator=create_model2_improved)

In [15]:
def create_model4_improved():
    input_delta = Input(shape=(4, 6), name='x_delta')
    input_abs_0 = Input(shape=(4, 9), name='x_abs_n_minus_1')
    input_abs = Input(shape=(4, 9), name='x_abs_n')
    input_force = Input(shape=(3, 1), name='x_force')

    input_abs_0_flat = Flatten()(input_abs_0)
    input_abs_flat = Flatten()(input_abs)
    input_force_flat = Flatten()(input_force)
    x = Concatenate()([input_abs_0_flat, input_abs_flat])
    x = Reshape((2, 4*9))(x)
    x = GRU(np.prod(dp.y_delta[0].shape) * 2, return_sequences=False,
    activation="tanh", recurrent_activation="sigmoid", recurrent_dropout=0, unroll=False, use_bias=True, reset_after=True
    )(x)
    x = Flatten()(x)
    x = Concatenate()([x, input_force_flat])
    x = Dropout(0.2)(x)
    x = Dense(np.prod(dp.y_delta[0].shape * 2), activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(np.prod(dp.y_delta[0].shape * 3), activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(np.prod(dp.y_delta[0].shape * 2), activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(np.prod(dp.y_delta[0].shape))(x)

    output_1 = Reshape(dp.y_delta[0].shape)(x)
    model = Model(inputs=[input_abs_0, input_abs, input_force], outputs=[output_1])
    model.compile(loss="mse", optimizer='rmsprop')
    return model

# 0.0092
# create_and_train_network(dp, dp.y_delta, 11, model_generator=create_model4_improved)

In [16]:
def create_model4_improved():
    input_delta = Input(shape=(4, 6), name='x_delta')
    input_abs_0 = Input(shape=(4, 9), name='x_abs_n_minus_1')
    input_abs = Input(shape=(4, 9), name='x_abs_n')
    input_force = Input(shape=(3, 1), name='x_force')

    input_abs_0_flat = Flatten()(input_abs_0)
    input_abs_flat = Flatten()(input_abs)
    input_force_flat = Flatten()(input_force)
    x = Concatenate()([input_abs_0_flat, input_abs_flat])
    x = Reshape((2, 4*9))(x)
    x = GRU(np.prod(dp.y_abs[0].shape * 2), return_sequences=False,
    activation="tanh", recurrent_activation="sigmoid", recurrent_dropout=0, unroll=False, use_bias=True, reset_after=True
    )(x)
    x = Flatten()(x)
    x = Concatenate()([x, input_force_flat])
    x = Dropout(0.2)(x)
    x = Dense(np.prod(dp.y_abs[0].shape * 2))(x)
    x = Dropout(0.2)(x)
    x = Dense(np.prod(dp.y_abs[0].shape * 2))(x)
    x = Dropout(0.2)(x)
    x = Dense(np.prod(dp.y_abs[0].shape))(x)

    output_1 = Reshape(dp.y_abs[0].shape)(x)
    model = Model(inputs=[input_abs_0, input_abs, input_force], outputs=[output_1])
    model.compile(loss="mse", optimizer='rmsprop')
    return model

#create_and_train_network(dp, dp.y_abs, 12, model_generator=create_model4_improved)

In [120]:
from tensorflow.keras.layers import BatchNormalization

def create_model2_simple():
    input_delta = Input(shape=(4, 6), name='x_delta')
    input_force = Input(shape=(3, 1), name='x_force')

    input_delta_flat = Flatten()(input_delta)
    input_force_flat = Flatten()(input_force)

    x = Concatenate()([input_delta_flat, input_force_flat])
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = BatchNormalization()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = BatchNormalization()(x)
    x = Dense(2048, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = BatchNormalization()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = BatchNormalization()(x)
    x = Dense(np.prod(dp.y_delta[1].shape))(x)

    output_1 = Reshape(dp.y_delta[1].shape)(x)
    model = Model(inputs=[input_delta, input_force], outputs=[output_1])
    model.compile(loss="mse", optimizer='rmsprop')
    return model
create_and_train_network(dp, dp.y_delta, 15, model_generator=create_model2_simple)

2021-03-07 09:59:10,206: Create model...
2021-03-07 09:59:10,271: Fit model..
/home/pawel/git/ueflow/venv/lib/python3.8/site-packages/tensorflow/python/keras/engine/functional.py:592: UserWarning: Input dict contained keys ['x_abs_n_minus_1', 'x_abs_n'] which did not match any model input. They will be ignored by the model.
  warnings.warn(


Model: "model_45"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
x_delta (InputLayer)            [(None, 4, 6)]       0                                            
__________________________________________________________________________________________________
x_force (InputLayer)            [(None, 3, 1)]       0                                            
__________________________________________________________________________________________________
flatten_331 (Flatten)           (None, 24)           0           x_delta[0][0]                    
__________________________________________________________________________________________________
flatten_332 (Flatten)           (None, 3)            0           x_force[0][0]                    
___________________________________________________________________________________________

2021-03-07 09:59:11,276: Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0031s vs `on_train_batch_end` time: 0.0053s). Check your callbacks.


385549/385560 [============================>.] - ETA: 0s - loss: 0.0090INFO:tensorflow:Assets written to: model\model_(428400, 4, 6)_(4, 6)_15_1615108702.2145362/assets


2021-03-07 10:18:23,486: Assets written to: model\model_(428400, 4, 6)_(4, 6)_15_1615108702.2145362/assets
/home/pawel/git/ueflow/venv/lib/python3.8/site-packages/tensorflow/python/keras/engine/functional.py:592: UserWarning: Input dict contained keys ['x_abs_n_minus_1', 'x_abs_n'] which did not match any model input. They will be ignored by the model.
  warnings.warn(


...

2021-03-07 10:57:56,881: Assets written to: model\model_(428400, 4, 6)_(4, 6)_15_1615111075.454998/assets


385560/385560 [==============================] - 1188s 3ms/step - loss: 0.0090 - val_loss: 16613919.0000

Epoch 00003: val_loss did not improve from 10617908.00000
Epoch 4/10
385558/385560 [============================>.] - ETA: 0s - loss: 0.0090INFO:tensorflow:Assets written to: model\model_(428400, 4, 6)_(4, 6)_15_1615112261.6153526/assets


2021-03-07 11:17:42,872: Assets written to: model\model_(428400, 4, 6)_(4, 6)_15_1615112261.6153526/assets


385560/385560 [==============================] - 1186s 3ms/step - loss: 0.0090 - val_loss: 12727600.0000

Epoch 00004: val_loss did not improve from 10617908.00000
Epoch 5/10
235685/385560 [=================>............] - ETA: 7:27 - loss: 0.0090

KeyboardInterrupt: 

In [94]:
def create_model4_improved():
    input_delta = Input(shape=(4, 6), name='x_delta')
    input_abs_0 = Input(shape=(4, 9), name='x_abs_n_minus_1')
    input_abs = Input(shape=(4, 9), name='x_abs_n')
    input_force = Input(shape=(3, 1), name='x_force')

    input_abs_0_flat = Flatten()(input_abs_0)
    input_abs_flat = Flatten()(input_abs)
    input_force_flat = Flatten()(input_force)
    input_delta_flat = Flatten()(input_delta)
    
    x = Concatenate()([input_abs_0, input_abs])
    #x = Reshape((2, 36))(x)
    x = GRU(np.prod(dp.y_abs[0].shape), return_sequences=True,
    activation="tanh", recurrent_activation="sigmoid", recurrent_dropout=0, unroll=False, use_bias=True, reset_after=True
    )(x)
    x = GRU(np.prod(dp.y_abs[0].shape * 2), return_sequences=False,
    activation="tanh", recurrent_activation="sigmoid", recurrent_dropout=0, unroll=False, use_bias=True, reset_after=True
    )(x)
    x = BatchNormalization()(x)
    x = Flatten()(x)
    x = Concatenate()([x, input_force_flat, input_delta_flat])
    x = Dropout(0.2)(x)
    x = Dense(np.prod(dp.y_delta[0].shape * 2))(x)
    x = Dropout(0.2)(x)
    x = Dense(np.prod(dp.y_delta[0].shape * 2))(x)
    x = Dropout(0.2)(x)
    x = Dense(np.prod(dp.y_delta[0].shape))(x)

    output_1 = Reshape(dp.y_delta[0].shape)(x)
    model = Model(inputs=[input_abs_0, input_abs, input_force, input_delta], outputs=[output_1])
    model.compile(loss="mse", optimizer='rmsprop')
    return model

create_and_train_network(dp, dp.y_delta, 12, model_generator=create_model4_improved)

2021-03-06 15:29:29,632: Create model...
2021-03-06 15:29:30,366: Fit model..


Model: "model_33"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
x_abs_n_minus_1 (InputLayer)    [(None, 4, 9)]       0                                            
__________________________________________________________________________________________________
x_abs_n (InputLayer)            [(None, 4, 9)]       0                                            
__________________________________________________________________________________________________
concatenate_102 (Concatenate)   (None, 4, 18)        0           x_abs_n_minus_1[0][0]            
                                                                 x_abs_n[0][0]                    
__________________________________________________________________________________________________
gru_29 (GRU)                    (None, 4, 36)        6048        concatenate_102[0][0]     

2021-03-06 15:29:31,621: Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0032s vs `on_train_batch_end` time: 0.0122s). Check your callbacks.


385560/385560 [==============================] - ETA: 0s - loss: 0.0085

2021-03-06 15:51:42,767: Found untraced functions such as gru_cell_29_layer_call_and_return_conditional_losses, gru_cell_29_layer_call_fn, gru_cell_30_layer_call_and_return_conditional_losses, gru_cell_30_layer_call_fn, gru_cell_29_layer_call_fn while saving (showing 5 of 10). These functions will not be directly callable after loading.
2021-03-06 15:51:42,819: Found untraced functions such as gru_cell_29_layer_call_and_return_conditional_losses, gru_cell_29_layer_call_fn, gru_cell_30_layer_call_and_return_conditional_losses, gru_cell_30_layer_call_fn, gru_cell_29_layer_call_fn while saving (showing 5 of 10). These functions will not be directly callable after loading.


...

2021-03-06 18:10:53,509: Found untraced functions such as gru_cell_29_layer_call_and_return_conditional_losses, gru_cell_29_layer_call_fn, gru_cell_30_layer_call_and_return_conditional_losses, gru_cell_30_layer_call_fn, gru_cell_29_layer_call_fn while saving (showing 5 of 10). These functions will not be directly callable after loading.
2021-03-06 18:10:53,561: Found untraced functions such as gru_cell_29_layer_call_and_return_conditional_losses, gru_cell_29_layer_call_fn, gru_cell_30_layer_call_and_return_conditional_losses, gru_cell_30_layer_call_fn, gru_cell_29_layer_call_fn while saving (showing 5 of 10). These functions will not be directly callable after loading.


INFO:tensorflow:Assets written to: model\model_(428400, 4, 6)_(4, 6)_12_1615050650.557672/assets


2021-03-06 18:10:54,087: Assets written to: model\model_(428400, 4, 6)_(4, 6)_12_1615050650.557672/assets


385560/385560 [==============================] - 1391s 4ms/step - loss: 0.0076 - val_loss: 0.5843

Epoch 00007: val_loss did not improve from 0.42971
Epoch 8/10
 97519/385560 [======>.......................] - ETA: 16:42 - loss: 0.0075

KeyboardInterrupt: 

In [93]:
from tensorflow.keras.layers import Add

def create_model4_improved():
    input_delta = Input(shape=(4, 6), name='x_delta')
    input_abs_0 = Input(shape=(4, 9), name='x_abs_n_minus_1')
    input_abs = Input(shape=(4, 9), name='x_abs_n')
    input_force = Input(shape=(3, 1), name='x_force')

    input_abs_0_flat = Flatten()(input_abs_0)
    input_abs_flat = Flatten()(input_abs)
    
    input_force_flat = Flatten()(input_force)
    input_delta_flat = Flatten()(input_delta)
    delta_force = Concatenate()([input_force_flat, input_delta_flat])
    delta_force = Dense(np.prod(dp.y_delta[0].shape))(delta_force)
    delta_force = Reshape(dp.y_delta[0].shape)(delta_force)
    
    x = Concatenate()([input_abs_0, input_abs])
    #x = Reshape((2, 36))(x)
    x = GRU(np.prod(dp.y_delta[0].shape), return_sequences=False,
    activation="tanh", recurrent_activation="sigmoid", recurrent_dropout=0, unroll=False, use_bias=True, reset_after=True
    )(x)
    x = Reshape(dp.y_delta[0].shape)(x)
    x = Add()([x, delta_force])
    x = Dense(np.prod(dp.y_delta[0].shape * 2))(x)
    x = Dropout(0.2)(x)
    x = Dense(dp.y_delta[0].shape[-1])(x)
    # print(x.shape, np.prod(dp.y_delta[0].shape), dp.y_delta[0].shape, "!!!!!!")
    output_1 = Reshape(dp.y_delta[0].shape)(x)
    model = Model(inputs=[input_abs_0, input_abs, input_force, input_delta], outputs=[output_1])
    model.compile(loss="mse", optimizer='rmsprop')
    return model

create_and_train_network(dp, dp.y_delta, 12, model_generator=create_model4_improved)

2021-03-06 13:18:05,964: Create model...
2021-03-06 13:18:06,084: Fit model..


Model: "model_32"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
x_force (InputLayer)            [(None, 3, 1)]       0                                            
__________________________________________________________________________________________________
x_delta (InputLayer)            [(None, 4, 6)]       0                                            
__________________________________________________________________________________________________
x_abs_n_minus_1 (InputLayer)    [(None, 4, 9)]       0                                            
__________________________________________________________________________________________________
x_abs_n (InputLayer)            [(None, 4, 9)]       0                                            
___________________________________________________________________________________________

2021-03-06 13:18:07,014: Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0013s vs `on_train_batch_end` time: 0.0057s). Check your callbacks.


385553/385560 [============================>.] - ETA: 0s - loss: 0.0046

2021-03-06 13:30:39,739: Found untraced functions such as gru_cell_28_layer_call_and_return_conditional_losses, gru_cell_28_layer_call_fn, gru_cell_28_layer_call_fn, gru_cell_28_layer_call_and_return_conditional_losses, gru_cell_28_layer_call_and_return_conditional_losses while saving (showing 5 of 5). These functions will not be directly callable after loading.
2021-03-06 13:30:39,773: Found untraced functions such as gru_cell_28_layer_call_and_return_conditional_losses, gru_cell_28_layer_call_fn, gru_cell_28_layer_call_fn, gru_cell_28_layer_call_and_return_conditional_losses, gru_cell_28_layer_call_and_return_conditional_losses while saving (showing 5 of 5). These functions will not be directly callable after loading.


...

2021-03-06 15:29:26,930: Summary



Epoch 00010: val_loss did not improve from 0.00276
Model: "model_32"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
x_force (InputLayer)            [(None, 3, 1)]       0                                            
__________________________________________________________________________________________________
x_delta (InputLayer)            [(None, 4, 6)]       0                                            
__________________________________________________________________________________________________
x_abs_n_minus_1 (InputLayer)    [(None, 4, 9)]       0                                            
__________________________________________________________________________________________________
x_abs_n (InputLayer)            [(None, 4, 9)]       0                                            
_______________________________________

2021-03-06 15:29:28,470: Found untraced functions such as gru_cell_28_layer_call_and_return_conditional_losses, gru_cell_28_layer_call_fn, gru_cell_28_layer_call_fn, gru_cell_28_layer_call_and_return_conditional_losses, gru_cell_28_layer_call_and_return_conditional_losses while saving (showing 5 of 5). These functions will not be directly callable after loading.
2021-03-06 15:29:28,505: Found untraced functions such as gru_cell_28_layer_call_and_return_conditional_losses, gru_cell_28_layer_call_fn, gru_cell_28_layer_call_fn, gru_cell_28_layer_call_and_return_conditional_losses, gru_cell_28_layer_call_and_return_conditional_losses while saving (showing 5 of 5). These functions will not be directly callable after loading.


INFO:tensorflow:Assets written to: /home/pawel/git/ueflow/samplesGeneration/logs/model_1615040966.933506/model/assets


2021-03-06 15:29:29,002: Assets written to: /home/pawel/git/ueflow/samplesGeneration/logs/model_1615040966.933506/model/assets


In [95]:
from tensorflow.keras.layers import Add

def create_model4_improved():
    input_delta = Input(shape=(4, 6), name='x_delta')
    input_abs_0 = Input(shape=(4, 9), name='x_abs_n_minus_1')
    input_abs = Input(shape=(4, 9), name='x_abs_n')
    input_force = Input(shape=(3, 1), name='x_force')

    input_abs_0_flat = Flatten()(input_abs_0)
    input_abs_flat = Flatten()(input_abs)
    
    input_force_flat = Flatten()(input_force)
    input_delta_flat = Flatten()(input_delta)
    delta_force = Concatenate()([input_force_flat, input_delta_flat])
    delta_force = Dense(np.prod(dp.y_delta[0].shape))(delta_force)
    delta_force = Reshape(dp.y_delta[0].shape)(delta_force)
    
    x = Concatenate()([input_abs_0, input_abs])
    #x = Reshape((2, 36))(x)
    x = GRU(np.prod(dp.y_delta[0].shape), return_sequences=False,
    activation="tanh", recurrent_activation="sigmoid", recurrent_dropout=0, unroll=False, use_bias=True, reset_after=True
    )(x)
    x = Reshape(dp.y_delta[0].shape)(x)
    x = Add()([x, delta_force])
    x = Dense(np.prod(dp.y_delta[0].shape * 2))(x)
    x = Dropout(0.2)(x)
    x = Dense(dp.y_abs[0].shape[-1])(x)
    # print(x.shape, np.prod(dp.y_delta[0].shape), dp.y_delta[0].shape, "!!!!!!")
    output_1 = Reshape(dp.y_abs[0].shape)(x)
    model = Model(inputs=[input_abs_0, input_abs, input_force, input_delta], outputs=[output_1])
    model.compile(loss="mse", optimizer='rmsprop')
    return model

create_and_train_network(dp, dp.y_abs, 12, model_generator=create_model4_improved)

2021-03-06 18:17:40,483: Create model...
2021-03-06 18:17:40,609: Fit model..


Model: "model_34"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
x_force (InputLayer)            [(None, 3, 1)]       0                                            
__________________________________________________________________________________________________
x_delta (InputLayer)            [(None, 4, 6)]       0                                            
__________________________________________________________________________________________________
x_abs_n_minus_1 (InputLayer)    [(None, 4, 9)]       0                                            
__________________________________________________________________________________________________
x_abs_n (InputLayer)            [(None, 4, 9)]       0                                            
___________________________________________________________________________________________

2021-03-06 18:17:41,357: Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0020s vs `on_train_batch_end` time: 0.0058s). Check your callbacks.


385542/385560 [============================>.] - ETA: 0s - loss: 0.3286

2021-03-06 18:30:19,139: Found untraced functions such as gru_cell_31_layer_call_and_return_conditional_losses, gru_cell_31_layer_call_fn, gru_cell_31_layer_call_fn, gru_cell_31_layer_call_and_return_conditional_losses, gru_cell_31_layer_call_and_return_conditional_losses while saving (showing 5 of 5). These functions will not be directly callable after loading.
2021-03-06 18:30:19,174: Found untraced functions such as gru_cell_31_layer_call_and_return_conditional_losses, gru_cell_31_layer_call_fn, gru_cell_31_layer_call_fn, gru_cell_31_layer_call_and_return_conditional_losses, gru_cell_31_layer_call_and_return_conditional_losses while saving (showing 5 of 5). These functions will not be directly callable after loading.


...

2021-03-06 19:37:03,087: Summary



Epoch 00006: val_loss did not improve from 0.05576
Model: "model_34"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
x_force (InputLayer)            [(None, 3, 1)]       0                                            
__________________________________________________________________________________________________
x_delta (InputLayer)            [(None, 4, 6)]       0                                            
__________________________________________________________________________________________________
x_abs_n_minus_1 (InputLayer)    [(None, 4, 9)]       0                                            
__________________________________________________________________________________________________
x_abs_n (InputLayer)            [(None, 4, 9)]       0                                            
_______________________________________

2021-03-06 19:37:04,830: Found untraced functions such as gru_cell_31_layer_call_and_return_conditional_losses, gru_cell_31_layer_call_fn, gru_cell_31_layer_call_fn, gru_cell_31_layer_call_and_return_conditional_losses, gru_cell_31_layer_call_and_return_conditional_losses while saving (showing 5 of 5). These functions will not be directly callable after loading.
2021-03-06 19:37:04,865: Found untraced functions such as gru_cell_31_layer_call_and_return_conditional_losses, gru_cell_31_layer_call_fn, gru_cell_31_layer_call_fn, gru_cell_31_layer_call_and_return_conditional_losses, gru_cell_31_layer_call_and_return_conditional_losses while saving (showing 5 of 5). These functions will not be directly callable after loading.


INFO:tensorflow:Assets written to: /home/pawel/git/ueflow/samplesGeneration/logs/model_1615055823.0924425/model/assets


2021-03-06 19:37:05,164: Assets written to: /home/pawel/git/ueflow/samplesGeneration/logs/model_1615055823.0924425/model/assets


In [97]:
from tensorflow.keras.layers import Add

def create_model4_improved():
    input_delta = Input(shape=(4, 6), name='x_delta')
    input_abs_0 = Input(shape=(4, 9), name='x_abs_n_minus_1')
    input_abs = Input(shape=(4, 9), name='x_abs_n')
    input_force = Input(shape=(3, 1), name='x_force')

    input_abs_0_flat = Flatten()(input_abs_0)
    input_abs_flat = Flatten()(input_abs)
    
    input_force_flat = Flatten()(input_force)
    input_delta_flat = Flatten()(input_delta)
    delta_force = Concatenate()([input_force_flat, input_delta_flat])
    delta_force = Dense(np.prod(dp.y_delta[0].shape))(delta_force)
    delta_force = Reshape(dp.y_delta[0].shape)(delta_force)
    
    x = Dense(np.prod(dp.y_delta[0].shape[-1]))(input_abs)
    x = Add()([x, delta_force])
    x = Dense(np.prod(128))(x)
    x = Dropout(0.2)(x)
    x = Dense(dp.y_delta[0].shape[-1])(x)
    # print(x.shape, np.prod(dp.y_delta[0].shape), dp.y_delta[0].shape, "!!!!!!")
    output_1 = Reshape(dp.y_delta[0].shape)(x)
    model = Model(inputs=[input_abs_0, input_abs, input_force, input_delta], outputs=[output_1])
    model.compile(loss="mse", optimizer='rmsprop')
    return model

create_and_train_network(dp, dp.y_delta, 12, model_generator=create_model4_improved)

2021-03-06 19:44:09,024: Create model...
2021-03-06 19:44:09,072: Fit model..


Model: "model_35"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
x_force (InputLayer)            [(None, 3, 1)]       0                                            
__________________________________________________________________________________________________
x_delta (InputLayer)            [(None, 4, 6)]       0                                            
__________________________________________________________________________________________________
flatten_263 (Flatten)           (None, 3)            0           x_force[0][0]                    
__________________________________________________________________________________________________
flatten_264 (Flatten)           (None, 24)           0           x_delta[0][0]                    
___________________________________________________________________________________________

2021-03-06 19:44:09,403: Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0012s vs `on_train_batch_end` time: 0.0019s). Check your callbacks.


385552/385560 [============================>.] - ETA: 0s - loss: 0.0103INFO:tensorflow:Assets written to: model\model_(428400, 4, 6)_(4, 6)_12_1615056710.3851628/assets


2021-03-06 19:51:51,144: Assets written to: model\model_(428400, 4, 6)_(4, 6)_12_1615056710.3851628/assets


...

2021-03-06 20:56:41,545: Assets written to: model\model_(428400, 4, 6)_(4, 6)_12_1615060600.9959114/assets


385560/385560 [==============================] - 488s 1ms/step - loss: 0.0028 - val_loss: 0.0028


2021-03-06 20:57:05,936: Summary



Epoch 00009: val_loss did not improve from 0.00264
Model: "model_35"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
x_force (InputLayer)            [(None, 3, 1)]       0                                            
__________________________________________________________________________________________________
x_delta (InputLayer)            [(None, 4, 6)]       0                                            
__________________________________________________________________________________________________
flatten_263 (Flatten)           (None, 3)            0           x_force[0][0]                    
__________________________________________________________________________________________________
flatten_264 (Flatten)           (None, 24)           0           x_delta[0][0]                    
_______________________________________

2021-03-06 20:57:06,479: Assets written to: /home/pawel/git/ueflow/samplesGeneration/logs/model_1615060625.9411633/model/assets


In [119]:
from tensorflow.keras.layers import Add

def create_model4_improved():
    input_delta = Input(shape=(4, 6), name='x_delta')
    input_abs_0 = Input(shape=(4, 9), name='x_abs_n_minus_1')
    input_abs = Input(shape=(4, 9), name='x_abs_n')
    input_force = Input(shape=(3, 1), name='x_force')

    input_abs_0_flat = Flatten()(input_abs_0)
    input_abs_flat = Flatten()(input_abs)
    
    input_force_flat = Flatten()(input_force)
    input_delta_flat = Flatten()(input_delta)
    delta_force = Concatenate()([input_force_flat, input_delta_flat])
    delta_force = Dense(np.prod(dp.y_delta[0].shape))(delta_force)
    delta_force = Reshape(dp.y_delta[0].shape)(delta_force)
    
    x = Dense(np.prod(dp.y_delta[0].shape[-1]))(input_abs)
    x = Reshape(dp.y_delta[0].shape)(x)
    x = Add()([x, delta_force])
    x = Dense(np.prod(dp.y_delta[0].shape * 2))(x)
    x = Dropout(0.2)(x)
    x = Dense(dp.y_abs[0].shape[-1])(x)
    # print(x.shape, np.prod(dp.y_delta[0].shape), dp.y_delta[0].shape, "!!!!!!")
    output_1 = Reshape(dp.y_abs[0].shape)(x)
    model = Model(inputs=[input_abs_0, input_abs, input_force, input_delta], outputs=[output_1])
    model.compile(loss="mse", optimizer='rmsprop')
    return model

create_and_train_network(dp, dp.y_abs, 12, model_generator=create_model4_improved)

2021-03-07 09:18:43,119: Create model...
2021-03-07 09:18:43,169: Fit model..


Model: "model_44"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
x_force (InputLayer)            [(None, 3, 1)]       0                                            
__________________________________________________________________________________________________
x_delta (InputLayer)            [(None, 4, 6)]       0                                            
__________________________________________________________________________________________________
flatten_329 (Flatten)           (None, 3)            0           x_force[0][0]                    
__________________________________________________________________________________________________
flatten_330 (Flatten)           (None, 24)           0           x_delta[0][0]                    
___________________________________________________________________________________________

2021-03-07 09:18:43,521: Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0014s vs `on_train_batch_end` time: 0.0019s). Check your callbacks.


385520/385560 [============================>.] - ETA: 0s - loss: 0.0559INFO:tensorflow:Assets written to: model\model_(428400, 4, 9)_(4, 9)_12_1615105584.311614/assets


2021-03-07 09:26:25,089: Assets written to: model\model_(428400, 4, 9)_(4, 9)_12_1615105584.311614/assets


...

2021-03-07 09:58:44,678: Assets written to: model\model_(428400, 4, 9)_(4, 9)_12_1615107524.0938792/assets


385560/385560 [==============================] - 489s 1ms/step - loss: 0.0386 - val_loss: 0.0108


2021-03-07 09:59:09,057: Summary



Epoch 00005: val_loss did not improve from 0.00928
Model: "model_44"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
x_force (InputLayer)            [(None, 3, 1)]       0                                            
__________________________________________________________________________________________________
x_delta (InputLayer)            [(None, 4, 6)]       0                                            
__________________________________________________________________________________________________
flatten_329 (Flatten)           (None, 3)            0           x_force[0][0]                    
__________________________________________________________________________________________________
flatten_330 (Flatten)           (None, 24)           0           x_delta[0][0]                    
_______________________________________

2021-03-07 09:59:09,664: Assets written to: /home/pawel/git/ueflow/samplesGeneration/logs/model_1615107549.0611713/model/assets


In [111]:
from tensorflow.keras.layers import Add

def create_model4_improved():
    input_delta = Input(shape=(4, 6), name='x_delta')
    input_abs_0 = Input(shape=(4, 9), name='x_abs_n_minus_1')
    input_abs = Input(shape=(4, 9), name='x_abs_n')
    input_force = Input(shape=(3, 1), name='x_force')

    input_abs_0_flat = Flatten()(input_abs_0)
    input_abs_flat = Flatten()(input_abs)
    
    input_force_flat = Flatten()(input_force)
    input_delta_flat = Flatten()(input_delta)
    delta_force = Concatenate()([input_force_flat, input_delta_flat])
    delta_force = Dense(np.prod(dp.y_delta[0].shape))(delta_force)
    delta_force = Reshape(dp.y_delta[0].shape)(delta_force)
    
    x = Concatenate()([input_abs_0, input_abs])
    #x = Reshape((2, 36))(x)
    x = GRU(np.prod(dp.y_delta[0].shape), return_sequences=False,
    activation="tanh", recurrent_activation="sigmoid", recurrent_dropout=0, unroll=False, use_bias=True, reset_after=True
    )(x)
    x = Reshape(dp.y_delta[0].shape)(x)
    x = Add()([x, delta_force])
    x = Dense(dp.y_abs[0].shape[-1] * 2)(x)
    x = Dropout(0.2)(x)
    x = Dense(dp.y_abs[0].shape[-1] * 4)(x)
    x = Dropout(0.2)(x)
    x = Dense(dp.y_abs[0].shape[-1] * 2)(x)
    x = Dropout(0.2)(x)
    x = Dense(dp.y_abs[0].shape[-1])(x)
    # print(x.shape, np.prod(dp.y_delta[0].shape), dp.y_delta[0].shape, "!!!!!!")
    output_1 = Reshape(dp.y_abs[0].shape)(x)
    model = Model(inputs=[input_abs_0, input_abs, input_force, input_delta], outputs=[output_1])
    model.compile(loss="mse", optimizer='rmsprop')
    return model

create_and_train_network(dp, dp.y_abs, 12, model_generator=create_model4_improved)

2021-03-06 21:09:16,464: Create model...
2021-03-06 21:09:16,606: Fit model..


Model: "model_40"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
x_force (InputLayer)            [(None, 3, 1)]       0                                            
__________________________________________________________________________________________________
x_delta (InputLayer)            [(None, 4, 6)]       0                                            
__________________________________________________________________________________________________
x_abs_n_minus_1 (InputLayer)    [(None, 4, 9)]       0                                            
__________________________________________________________________________________________________
x_abs_n (InputLayer)            [(None, 4, 9)]       0                                            
___________________________________________________________________________________________

2021-03-06 21:09:17,658: Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0023s vs `on_train_batch_end` time: 0.0068s). Check your callbacks.


385541/385560 [============================>.] - ETA: 0s - loss: 1.4347

2021-03-06 21:24:29,418: Found untraced functions such as gru_cell_32_layer_call_and_return_conditional_losses, gru_cell_32_layer_call_fn, gru_cell_32_layer_call_fn, gru_cell_32_layer_call_and_return_conditional_losses, gru_cell_32_layer_call_and_return_conditional_losses while saving (showing 5 of 5). These functions will not be directly callable after loading.
2021-03-06 21:24:29,466: Found untraced functions such as gru_cell_32_layer_call_and_return_conditional_losses, gru_cell_32_layer_call_fn, gru_cell_32_layer_call_fn, gru_cell_32_layer_call_and_return_conditional_losses, gru_cell_32_layer_call_and_return_conditional_losses while saving (showing 5 of 5). These functions will not be directly callable after loading.


...

2021-03-06 22:27:29,770: Found untraced functions such as gru_cell_32_layer_call_and_return_conditional_losses, gru_cell_32_layer_call_fn, gru_cell_32_layer_call_fn, gru_cell_32_layer_call_and_return_conditional_losses, gru_cell_32_layer_call_and_return_conditional_losses while saving (showing 5 of 5). These functions will not be directly callable after loading.
2021-03-06 22:27:29,814: Found untraced functions such as gru_cell_32_layer_call_and_return_conditional_losses, gru_cell_32_layer_call_fn, gru_cell_32_layer_call_fn, gru_cell_32_layer_call_and_return_conditional_losses, gru_cell_32_layer_call_and_return_conditional_losses while saving (showing 5 of 5). These functions will not be directly callable after loading.


INFO:tensorflow:Assets written to: model\model_(428400, 4, 9)_(4, 9)_12_1615066047.850196/assets


2021-03-06 22:27:30,156: Assets written to: model\model_(428400, 4, 9)_(4, 9)_12_1615066047.850196/assets


385560/385560 [==============================] - 949s 2ms/step - loss: 1.3185 - val_loss: 0.3805

Epoch 00005: val_loss did not improve from 0.30346
Epoch 6/10
270307/385560 [====================>.........] - ETA: 4:31 - loss: 1.3495

KeyboardInterrupt: 

In [118]:
from tensorflow.keras.layers import Add

def create_model4_improved():
    input_delta = Input(shape=(4, 6), name='x_delta')
    input_abs_0 = Input(shape=(4, 9), name='x_abs_n_minus_1')
    input_abs = Input(shape=(4, 9), name='x_abs_n')
    input_force = Input(shape=(3, 1), name='x_force')

    input_abs_0_flat = Flatten()(input_abs_0)
    input_abs_flat = Flatten()(input_abs)
    input_force_flat = Flatten()(input_force)
    input_delta_flat = Flatten()(input_delta)
    
    input_force_part = Concatenate()([input_abs_flat, input_force_flat])
    input_force_part = Dense(np.prod(dp.y_delta[0].shape))(input_force_part)
    
    
    x = Add()([input_delta_flat, input_force_part])
    x = Reshape(dp.y_delta[0].shape)(x)
    x = Dense(256)(x)
    x = Dense(512)(x)
    x = Dense(512)(x)
    x = Dense(256)(x)
    x = Dense(dp.y_delta[0].shape[-1])(x)
    # print(x.shape, np.prod(dp.y_delta[0].shape), dp.y_delta[0].shape, "!!!!!!")
    output_1 = Reshape(dp.y_delta[0].shape)(x)
    model = Model(inputs=[input_abs_0, input_abs, input_force, input_delta], outputs=[output_1])
    model.compile(loss="mse", optimizer='rmsprop')
    return model

create_and_train_network(dp, dp.y_delta, 12, model_generator=create_model4_improved)

2021-03-07 08:01:22,031: Create model...
2021-03-07 08:01:22,089: Fit model..


Model: "model_43"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
x_abs_n (InputLayer)            [(None, 4, 9)]       0                                            
__________________________________________________________________________________________________
x_force (InputLayer)            [(None, 3, 1)]       0                                            
__________________________________________________________________________________________________
flatten_324 (Flatten)           (None, 36)           0           x_abs_n[0][0]                    
__________________________________________________________________________________________________
flatten_325 (Flatten)           (None, 3)            0           x_force[0][0]                    
___________________________________________________________________________________________

2021-03-07 08:01:22,483: Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0017s vs `on_train_batch_end` time: 0.0022s). Check your callbacks.


385555/385560 [============================>.] - ETA: 0s - loss: 0.0122INFO:tensorflow:Assets written to: model\model_(428400, 4, 6)_(4, 6)_12_1615101032.0964904/assets


2021-03-07 08:10:33,012: Assets written to: model\model_(428400, 4, 6)_(4, 6)_12_1615101032.0964904/assets


...

2021-03-07 09:18:16,094: Assets written to: model\model_(428400, 4, 6)_(4, 6)_12_1615105095.4306498/assets


385560/385560 [==============================] - 580s 2ms/step - loss: 0.0054 - val_loss: 0.0040


2021-03-07 09:18:41,858: Summary



Epoch 00008: val_loss did not improve from 0.00384
Model: "model_43"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
x_abs_n (InputLayer)            [(None, 4, 9)]       0                                            
__________________________________________________________________________________________________
x_force (InputLayer)            [(None, 3, 1)]       0                                            
__________________________________________________________________________________________________
flatten_324 (Flatten)           (None, 36)           0           x_abs_n[0][0]                    
__________________________________________________________________________________________________
flatten_325 (Flatten)           (None, 3)            0           x_force[0][0]                    
_______________________________________

2021-03-07 09:18:42,557: Assets written to: /home/pawel/git/ueflow/samplesGeneration/logs/model_1615105121.8618183/model/assets


In [114]:
from tensorflow.keras.layers import Add

def create_model4_improved():
    input_delta = Input(shape=(4, 6), name='x_delta')
    input_abs_0 = Input(shape=(4, 9), name='x_abs_n_minus_1')
    input_abs = Input(shape=(4, 9), name='x_abs_n')
    input_force = Input(shape=(3, 1), name='x_force')

    input_abs_0_flat = Flatten()(input_abs_0)
    input_abs_flat = Flatten()(input_abs)
    
    input_force_flat = Flatten()(input_force)
    input_delta_flat = Flatten()(input_delta)
    
    input_force_part = Concatenate()([input_abs_flat, input_force_flat])
    input_force_part = Dense(np.prod(dp.y_delta[0].shape))(input_force_part)
    
    
    x = Add()([input_delta_flat, input_force_part])
    x = Dense(256)(x)
    x = Dense(512)(x)
    x = Dense(512)(x)
    x = Dense(256)(x)
    x = Dense(np.prod(dp.y_abs[0].shape))(x)
    # print(x.shape, np.prod(dp.y_delta[0].shape), dp.y_delta[0].shape, "!!!!!!")
    output_1 = Reshape(dp.y_abs[0].shape)(x)
    model = Model(inputs=[input_abs_0, input_abs, input_force, input_delta], outputs=[output_1])
    model.compile(loss="mse", optimizer='rmsprop')
    return model

create_and_train_network(dp, dp.y_abs, 12, model_generator=create_model4_improved)

2021-03-07 07:47:13,493: Create model...
2021-03-07 07:47:13,524: Fit model..


Model: "model_42"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
x_abs_n (InputLayer)            [(None, 4, 9)]       0                                            
__________________________________________________________________________________________________
x_force (InputLayer)            [(None, 3, 1)]       0                                            
__________________________________________________________________________________________________
flatten_310 (Flatten)           (None, 36)           0           x_abs_n[0][0]                    
__________________________________________________________________________________________________
flatten_311 (Flatten)           (None, 3)            0           x_force[0][0]                    
___________________________________________________________________________________________

2021-03-07 07:47:13,903: Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0016s vs `on_train_batch_end` time: 0.0022s). Check your callbacks.


385538/385560 [============================>.] - ETA: 0s - loss: 0.1294INFO:tensorflow:Assets written to: model\model_(428400, 4, 9)_(4, 9)_12_1615100174.3266542/assets


2021-03-07 07:56:14,810: Assets written to: model\model_(428400, 4, 9)_(4, 9)_12_1615100174.3266542/assets


385560/385560 [==============================] - 567s 1ms/step - loss: 0.1294 - val_loss: 0.0170

Epoch 00001: val_loss improved from inf to 0.01700, saving model to (428400, 4, 9)_(4, 9)_12_model.h5
Epoch 2/10
 22908/385560 [>.............................] - ETA: 8:21 - loss: 0.0172

KeyboardInterrupt: 

In [112]:
from tensorflow.keras.layers import Add

def create_model4_improved():
    input_delta = Input(shape=(4, 6), name='x_delta')
    input_abs_0 = Input(shape=(4, 9), name='x_abs_n_minus_1')
    input_abs = Input(shape=(4, 9), name='x_abs_n')
    input_force = Input(shape=(3, 1), name='x_force')

    input_abs_0_flat = Flatten()(input_abs_0)
    input_abs_flat = Flatten()(input_abs)
    input_abs_rs = Dense(16)(input_abs)
    input_abs_rs = Dense(32)(input_abs_rs)
    input_abs_rs = Dense(8)(input_abs_rs)
    input_abs_rs = Dense(6)(input_abs_rs)
    
    input_force_flat = Flatten()(input_force)
    input_delta_flat = Flatten()(input_delta)
    
    input_force_part = Concatenate()([input_abs_flat, input_force_flat])
    input_force_part = Dense(np.prod(dp.y_delta[0].shape))(input_force_part)
    
    
    x = Add()([input_delta_flat, input_force_part])
    x = Reshape(dp.y_delta[0].shape)(x)
    x = Add()([x, input_abs_rs])
    x = Dense(256)(x)
    x = Dense(512)(x)
    x = Dense(512)(x)
    x = Dense(256)(x)
    x = Dense(dp.y_abs[0].shape[-1])(x)
    # print(x.shape, np.prod(dp.y_delta[0].shape), dp.y_delta[0].shape, "!!!!!!")
    output_1 = Reshape(dp.y_abs[0].shape)(x)
    model = Model(inputs=[input_abs_0, input_abs, input_force, input_delta], outputs=[output_1])
    model.compile(loss="mse", optimizer='rmsprop')
    return model

create_and_train_network(dp, dp.y_abs, 12, model_generator=create_model4_improved)

2021-03-06 22:38:53,418: Create model...
2021-03-06 22:38:53,510: Fit model..


Model: "model_41"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
x_abs_n (InputLayer)            [(None, 4, 9)]       0                                            
__________________________________________________________________________________________________
x_force (InputLayer)            [(None, 3, 1)]       0                                            
__________________________________________________________________________________________________
flatten_304 (Flatten)           (None, 36)           0           x_abs_n[0][0]                    
__________________________________________________________________________________________________
flatten_305 (Flatten)           (None, 3)            0           x_force[0][0]                    
___________________________________________________________________________________________

2021-03-06 22:38:54,077: Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0018s vs `on_train_batch_end` time: 0.0039s). Check your callbacks.


385538/385560 [============================>.] - ETA: 0s - loss: 0.1168INFO:tensorflow:Assets written to: model\model_(428400, 4, 9)_(4, 9)_12_1615067534.6099675/assets


2021-03-06 22:52:15,789: Assets written to: model\model_(428400, 4, 9)_(4, 9)_12_1615067534.6099675/assets


...

2021-03-07 00:01:31,200: Assets written to: model\model_(428400, 4, 9)_(4, 9)_12_1615071690.1964667/assets


385560/385560 [==============================] - 831s 2ms/step - loss: 0.0124 - val_loss: 0.0150


2021-03-07 00:02:00,776: Summary



Epoch 00006: val_loss did not improve from 0.00925
Model: "model_41"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
x_abs_n (InputLayer)            [(None, 4, 9)]       0                                            
__________________________________________________________________________________________________
x_force (InputLayer)            [(None, 3, 1)]       0                                            
__________________________________________________________________________________________________
flatten_304 (Flatten)           (None, 36)           0           x_abs_n[0][0]                    
__________________________________________________________________________________________________
flatten_305 (Flatten)           (None, 3)            0           x_force[0][0]                    
_______________________________________

2021-03-07 00:02:01,924: Assets written to: /home/pawel/git/ueflow/samplesGeneration/logs/model_1615071720.778867/model/assets


In [124]:
from tensorflow.keras.layers import Add

def create_model4_improved():
    input_delta = Input(shape=(4, 6), name='x_delta')
    input_abs_0 = Input(shape=(4, 9), name='x_abs_n_minus_1')
    input_abs = Input(shape=(4, 9), name='x_abs_n')
    input_force = Input(shape=(3, 1), name='x_force')

    input_abs_0_flat = Flatten()(input_abs_0)
    input_abs_flat = Flatten()(input_abs)
    input_abs_rs = Dense(16)(input_abs)
    input_abs_rs = Dense(32)(input_abs_rs)
    input_abs_rs = Dense(8)(input_abs_rs)
    input_abs_rs = Dense(6)(input_abs_rs)
    
    input_force_flat = Flatten()(input_force)
    input_delta_flat = Flatten()(input_delta)
    
    input_force_part = Concatenate()([input_abs_flat, input_force_flat])
    input_force_part = Dense(512)(input_force_part)
    input_force_part = Dense(np.prod(dp.y_delta[0].shape))(input_force_part)
    
    
    x = Add()([input_delta_flat, input_force_part])
    x = Reshape(dp.y_delta[0].shape)(x)
    x = Add()([x, input_abs_rs])
    x = Dense(256)(x)
    x = Dense(512)(x)
    x = Dense(512)(x)
    x = Dense(256)(x)
    x = Dense(dp.y_abs[0].shape[-1])(x)
    # print(x.shape, np.prod(dp.y_delta[0].shape), dp.y_delta[0].shape, "!!!!!!")
    output_1 = Reshape(dp.y_abs[0].shape)(x)
    model = Model(inputs=[input_abs_0, input_abs, input_force, input_delta], outputs=[output_1])
    model.compile(loss="mse", optimizer='rmsprop')
    return model

create_and_train_network(dp, dp.y_abs, 12, model_generator=create_model4_improved)

2021-03-07 12:38:01,741: Create model...
2021-03-07 12:38:01,845: Fit model..


Model: "model_48"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
x_abs_n (InputLayer)            [(None, 4, 9)]       0                                            
__________________________________________________________________________________________________
x_force (InputLayer)            [(None, 3, 1)]       0                                            
__________________________________________________________________________________________________
flatten_346 (Flatten)           (None, 36)           0           x_abs_n[0][0]                    
__________________________________________________________________________________________________
flatten_347 (Flatten)           (None, 3)            0           x_force[0][0]                    
___________________________________________________________________________________________

2021-03-07 12:38:02,447: Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0024s vs `on_train_batch_end` time: 0.0038s). Check your callbacks.


385556/385560 [============================>.] - ETA: 0s - loss: 0.1471INFO:tensorflow:Assets written to: model\model_(428400, 4, 9)_(4, 9)_12_1615117953.1743207/assets


2021-03-07 12:52:34,343: Assets written to: model\model_(428400, 4, 9)_(4, 9)_12_1615117953.1743207/assets


...

2021-03-07 14:21:53,803: Assets written to: model\model_(428400, 4, 9)_(4, 9)_12_1615123312.7563915/assets


385560/385560 [==============================] - 895s 2ms/step - loss: 0.0164 - val_loss: 0.0982


2021-03-07 14:22:24,361: Summary



Epoch 00007: val_loss did not improve from 0.01498
Model: "model_48"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
x_abs_n (InputLayer)            [(None, 4, 9)]       0                                            
__________________________________________________________________________________________________
x_force (InputLayer)            [(None, 3, 1)]       0                                            
__________________________________________________________________________________________________
flatten_346 (Flatten)           (None, 36)           0           x_abs_n[0][0]                    
__________________________________________________________________________________________________
flatten_347 (Flatten)           (None, 3)            0           x_force[0][0]                    
_______________________________________

2021-03-07 14:22:25,607: Assets written to: /home/pawel/git/ueflow/samplesGeneration/logs/model_1615123344.3664703/model/assets
